# GBO ARGUS spectra dissection



In [ ]:
%matplotlib inline

from astropy.io import fits
from astropy import units as u
import numpy as np
from matplotlib import pyplot as plt
from astropy.visualization import quantity_support
from specutils import Spectrum1D, SpectrumList

from astropy.io import ascii
from astropy.nddata import StdDevUncertainty
from astropy.table import Table
from astropy.units import Unit
from astropy.wcs import WCS
from astropy.convolution import convolve, Box1DKernel

from specutils.io import get_loaders_by_extension
from specutils.io.registers import data_loader
from specutils import Spectrum1D

## SDFITS

We begin by dissecting the typical SDFITS file, starting with raw plotting of a spectrum and some basic BINTABLE operations.

The data in a bintable has **rows** and **columns**.   Within GBTIDL a **row** is sometimes also referred to as a **record** and sometimes **#index**.

## First a few useful functions

In [ ]:
def my_stats(label,data,edge=0):
    """
    display mean,rms,min,max,npts
    also good for regression
    can optionally take some edges of either side (check)
    """
    if edge > 0:
        mean = data[edge:-edge].mean()
        rms  = data[edge:-edge].std()
        dmin = data[edge:-edge].min()
        dmax = data[edge:-edge].max()
    else:
        mean = data.mean()
        rms  = data.std()
        dmin = data.min()
        dmax = data.max()
    print("%s  %s %s %s %s %d" %  (label,repr(mean),repr(rms),repr(dmin),repr(dmax),len(data)-2*edge))

In [ ]:
def uniq(seq):
    """ from http://stackoverflow.com/questions/480214/how-do-you-remove-duplicates-from-a-list-in-python-whilst-preserving-order """
    seen = set()
    seen_add = seen.add
    return [ x for x in seq if x not in seen and not seen_add(x)]


## Input parameters

define the SDFITS file name 

In [ ]:
fname = 'AGBT17B_151_01.raw.vegas.A.fits'  #  72MB argus 1/8

Open the FITS file and point to the 2nd HDU, where the BINTABLE is located. No error checking. Data isn't really read yet.

The DATA for argus comes with two additional sources, the VANE and the SKY, in addition to our source, IC0342 in this case. They are used for additional calibration. Also, this is one of 8 files (labeles A through H), but for the purpose of this experiment, A will suffice. A contains two of the beams (pointings), all filed will thus contain the full 16 pointings.

In [ ]:
%%time
# 20ms: for ex1  fast because data is not really put in memory?
# 40ms for EDGE
nrow=0
try:
    hdu = fits.open(fname)
    header2 = hdu[1].header
    data2   = hdu[1].data
    nrow = len(data2)
    print("Found %d rows in %s" % (nrow,fname))
except:
    print("*** Error ***")

Lets stat all the numbers (nrow * nchan)

In [ ]:
%%time
# 790ms for ex1
my_stats(fname,data2[:]['DATA'])

Grab few spectra by row numbers.

In [ ]:
%%time 
#  3ms
row     = [0,40,80]
spectra = data2[:]['DATA']  
nchan   = len(spectra[0])
chans   = np.arange(nchan)
src     = data2[row]['OBJECT']
print("Found %d channels and object %s" % (nchan,src))
#
my_stats('STATS for row',spectra[row])

A super simple plot, channel number vs. flux for the selected rows

In [ ]:
for i in range(len(row)):
    plt.plot(chans,spectra[row[i]],label=src[i])
plt.legend()
plt.xlabel("Channel")
plt.ylabel("Flux");
plt.title("Raw Spectrum %s - rows %s" % (src,repr(row)));


### IC0342
From now on, we will be working with just the source itself

In [ ]:
%%time 
whsrc = data2[:]['OBJECT'] == 'IC0342'
data3 = data2[whsrc]
spectra = data3[:]['DATA']
print(data2.shape, data3.shape)

Show a waterfall plot for a select number of rows. 

In [ ]:
print(spectra.shape)
plt.imshow(spectra[0:1024],origin='bottom')
plt.xlabel('Channel')
plt.ylabel('Row')
plt.title("section of the waterfall plot");

The rows are organized in by their SCAN number. Each scan has a uniq TRGTLONG and TRGTLAT

In [ ]:
scans = data3['SCAN']
uscans = uniq(scans)
print("Found %d scans" % len(uscans))
nrows = len(data3)
scanlen = nrows // len(uscans)
print(uscans)
print(nrows,scanlen)

In [ ]:
rows = list(range(0,nrows,scanlen))
tra = data3[rows]['TRGTLONG']
tdec= data3[rows]['TRGTLAT']
for s in uscans:
    wh = scans == s
    ra = data3[wh]['CRVAL2']
    dec = data3[wh]['CRVAL3']
    plt.plot(ra,dec,'.')
plt.plot(tra,tdec,'o',color='black',fillstyle='full')
plt.title("All %d scans" % len(uscans))
plt.xlabel("RA")
plt.ylabel("DEC")